In [1]:
import pandas as pd
import nltk
import time
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ccompain/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df_SampleCompany1 = pd.read_csv("./SampleData/Sample_Company1.csv")
df_SampleCompany2 = pd.read_csv("./SampleData/Sample_Company2.csv")

In [3]:
df_SampleCompany1.head(5)

,id,title,description,manufacturer,price
0,b0000aka82,studyworks! teaching pro: middle school math &...,studyworks teaching pro middle school math & s...,global software publishing,29.99
1,b000bnb72g,movies on psp,x-zoom movies on psp is the first commercial v...,x-oom,0.00
2,b0002e3g6o,eastwest bsendorfer 290 grand piano virtual in...,the 5-star awarded pmi bdorfer 290 is simply t...,eastwest,199.95
3,b00008ajjc,passwords plus 1.0,passwords plus stores all your personal inform...,dataviz,29.99
4,b0006g2wke,microsoft licenses project svr sa govt (h2200301),microsoft project server win32 english softwar...,microsoft licenses,3601.40


In [4]:
df_SampleCompany2.head(5)


,id,name,description,manufacturer,price
0,12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99
1,5781318040297863663,ipswitch wd-1000-0900 ws_ftp pro 9 sdk w/ ws_f...,no description available,ipswitch,325.51
2,9949258255997064102,money prem 2007 cd minibox,money premium 2007 win32 english north america...,microsoft,63.99
3,9761533219806554318,adobe indesign cs3 for mac upgrade,system requirements powerpc g4 or g5 or intel ...,NaN,205.99
4,18226456193742595288,bamboo technology llc vault360 combo,capitalizing on the popularity of the ipod psp...,NaN,24.58


In [5]:
col_names = ['token_Company1','token_Company2','jd']

matches = []
matchescomplet = []

number_of_matches = 0
tokens1=[]
tokens2=[]

df_score = pd.DataFrame()

for i in range(len(df_SampleCompany1)):
    tokens1name = nltk.word_tokenize(df_SampleCompany1.iloc[i,1])
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    #print(ng1_tokensname)
    for j in range(len(df_SampleCompany2)):
        tokens2name = nltk.word_tokenize(df_SampleCompany2.iloc[j,1])
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
        name_score = nltk.edit_distance(df_SampleCompany1.iloc[i,1], df_SampleCompany2.iloc[j,1])
        if (jd_ng1_ng2_name <= 0.75) or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((df_SampleCompany1.iloc[i,0],df_SampleCompany2.iloc[j,0]))

print("Number of matches: {}".format(number_of_matches))

Number of matches: 103


In [10]:
col_names = ['token_Company1','token_Company2','jd']

matches = []
matchescomplet = []

number_of_matches = 0
tokens1=[]
tokens2=[]

df_score = pd.DataFrame()

for i in range(len(df_SampleCompany1)):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    tokens1name = tokenizer.tokenize(df_SampleCompany1.iloc[i,1])
    ng1_tokensname = set(nltk.ngrams(tokens1name, n=1))
    for j in range(len(df_SampleCompany2)):
        tokens2name = tokenizer.tokenize(df_SampleCompany2.iloc[j,1])
        ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
        jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname,ng2_tokensname)
        name_score = nltk.edit_distance(df_SampleCompany1.iloc[i,1], df_SampleCompany2.iloc[j,1])
        if (jd_ng1_ng2_name <= 0.70) or name_score<=1 :
            number_of_matches = number_of_matches +1
            matches.append((df_SampleCompany1.iloc[i,0],df_SampleCompany2.iloc[j,0]))

print("Number of matches: {}".format(number_of_matches))


Number of matches: 101


In [19]:
ground_truth_matches = pd.read_csv("./SampleData/Sample_Groud_truth_mappings.csv")
#ground_truth_matches.insert(0, 'record_ID', range(0, len(ground_truth_matches)))
ground_truth_matches.head(5)

,idCompany1,idCompany2
0,b0002ibev4,1887899244694755891
1,b0007lw22g,18398718226932431716
2,b000ofnri8,12244614697089679523
3,b0007d8r5k,13775362651326388438
4,b00099qrok,9755705822363275907


In [20]:
matches_df = pd.DataFrame(matches)
matches_df.columns= ['idCompany1','idCompany2']
matches_df.head(5)

,idCompany1,idCompany2
0,b0000aka82,15883771067913561390
1,b000bnb72g,17010841797296818807
2,b00008ajjc,18424464601572004274
3,b0006g2wke,18422068306669698354
4,b0000c6fjm,18413075895200367717


In [26]:
diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
diff_df.head(5)

,idCompany1,idCompany2,Exist
0,b0002ibev4,1887899244694755891,both
1,b0007lw22g,18398718226932431716,both
2,b000ofnri8,12244614697089679523,left_only
3,b0007d8r5k,13775362651326388438,left_only
4,b00099qrok,9755705822363275907,both


In [25]:
true_positives = diff_df[diff_df.Exist=='both']
false_positives = diff_df[diff_df.Exist=='right_only']
false_negatives = diff_df[diff_df.Exist=='left_only']

precision = len(true_positives)/(len(true_positives)+ len(false_positives))
recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
f_measure = 2*(precision*recall)/(precision+recall)

print(f"precision : {precision}")
print(f"recall : {recall}")
print(f"f_measure : {f_measure}")

precision : 0.8415841584158416
recall : 0.8173076923076923
f_measure : 0.8292682926829268
